In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

# Datasets load

In [2]:
def load_mnist():
    import os
    path = os.path.join(os.getcwd(), 'mnist.npz')
    
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data(path)
    
    # [batch_size, height, width, color_channel]
    train_data = np.expand_dims(train_data, axis=-1)
    test_data = np.expand_dims(test_data, axis=-1)
    
    # data normalization
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    # One hot encoding
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    
    return train_data, train_labels, test_data, test_labels

# Create model (activation function = sigmoid)

In [3]:
class CreateModel(tf.keras.Model):
    def __init__(self, label_dims):
        super(CreateModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=256, use_bias=True, kernel_initializer='random_normal')
        self.sigmoid1 = tf.keras.layers.Activation(tf.keras.activations.sigmoid)
        self.dense2 = tf.keras.layers.Dense(units=512, use_bias=True, kernel_initializer='random_normal')
        self.sigmoid2 = tf.keras.layers.Activation(tf.keras.activations.sigmoid)
        self.dense3 = tf.keras.layers.Dense(units=label_dims, use_bias=True, kernel_initializer='random_normal')
    
    def call(self, inputs, training=False):
        net = self.flatten(inputs)
        net = self.dense1(net)
        net = self.sigmoid1(net)
        net = self.dense2(net)
        net = self.sigmoid2(net)
        net = self.dense3(net)
        
        return net

# Loss 정의하기

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))
    return loss
    
def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32)) * 100
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# 데이터 전처리

In [5]:
train_x, train_y, test_x, test_y = load_mnist()

learning_rate = 0.001
batch_size = 128
train_dataset_num = len(train_x) // batch_size

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x), drop_remainder=True)

# 학습 하기

In [6]:
label_dim = 10

model = CreateModel(label_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

epoch = 1
for i in range(epoch):
    for idx, (images, labels) in enumerate(train_dataset):
        grads = grad(model, images, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        
        loss = loss_fn(model, images, labels)
        accuracy = accuracy_fn(model, images, labels)
        
        print('Epoch: {:5}, Process: {:5}/{}, Loss: {:5.5f}, Accuracy: {:5.1f}%'.format(i, idx, train_dataset_num, loss, accuracy))

Epoch:     0, Process:     0/468, Loss: 2.25441, Accuracy:  14.8%
Epoch:     0, Process:     1/468, Loss: 2.28027, Accuracy:  13.3%
Epoch:     0, Process:     2/468, Loss: 2.34843, Accuracy:  10.9%
Epoch:     0, Process:     3/468, Loss: 2.22974, Accuracy:  22.7%
Epoch:     0, Process:     4/468, Loss: 2.25883, Accuracy:  12.5%
Epoch:     0, Process:     5/468, Loss: 2.19500, Accuracy:  20.3%
Epoch:     0, Process:     6/468, Loss: 2.23685, Accuracy:  28.1%
Epoch:     0, Process:     7/468, Loss: 2.31854, Accuracy:  18.0%
Epoch:     0, Process:     8/468, Loss: 2.18890, Accuracy:  24.2%
Epoch:     0, Process:     9/468, Loss: 2.18071, Accuracy:  27.3%
Epoch:     0, Process:    10/468, Loss: 2.13765, Accuracy:  44.5%
Epoch:     0, Process:    11/468, Loss: 2.15558, Accuracy:  42.2%
Epoch:     0, Process:    12/468, Loss: 2.11137, Accuracy:  32.0%
Epoch:     0, Process:    13/468, Loss: 2.07785, Accuracy:  20.3%
Epoch:     0, Process:    14/468, Loss: 2.05313, Accuracy:  30.5%
Epoch:    

Epoch:     0, Process:   128/468, Loss: 0.44488, Accuracy:  92.2%
Epoch:     0, Process:   129/468, Loss: 0.48287, Accuracy:  86.7%
Epoch:     0, Process:   130/468, Loss: 0.41179, Accuracy:  91.4%
Epoch:     0, Process:   131/468, Loss: 0.31498, Accuracy:  94.5%
Epoch:     0, Process:   132/468, Loss: 0.53630, Accuracy:  84.4%
Epoch:     0, Process:   133/468, Loss: 0.60965, Accuracy:  85.2%
Epoch:     0, Process:   134/468, Loss: 0.47906, Accuracy:  85.9%
Epoch:     0, Process:   135/468, Loss: 0.50184, Accuracy:  86.7%
Epoch:     0, Process:   136/468, Loss: 0.43031, Accuracy:  89.1%
Epoch:     0, Process:   137/468, Loss: 0.36676, Accuracy:  91.4%
Epoch:     0, Process:   138/468, Loss: 0.40382, Accuracy:  87.5%
Epoch:     0, Process:   139/468, Loss: 0.40905, Accuracy:  90.6%
Epoch:     0, Process:   140/468, Loss: 0.39539, Accuracy:  90.6%
Epoch:     0, Process:   141/468, Loss: 0.53329, Accuracy:  86.7%
Epoch:     0, Process:   142/468, Loss: 0.41342, Accuracy:  88.3%
Epoch:    

Epoch:     0, Process:   253/468, Loss: 0.23421, Accuracy:  92.2%
Epoch:     0, Process:   254/468, Loss: 0.31378, Accuracy:  89.1%
Epoch:     0, Process:   255/468, Loss: 0.29612, Accuracy:  89.8%
Epoch:     0, Process:   256/468, Loss: 0.25489, Accuracy:  91.4%
Epoch:     0, Process:   257/468, Loss: 0.23305, Accuracy:  93.0%
Epoch:     0, Process:   258/468, Loss: 0.24230, Accuracy:  91.4%
Epoch:     0, Process:   259/468, Loss: 0.34893, Accuracy:  90.6%
Epoch:     0, Process:   260/468, Loss: 0.39064, Accuracy:  93.0%
Epoch:     0, Process:   261/468, Loss: 0.27465, Accuracy:  93.0%
Epoch:     0, Process:   262/468, Loss: 0.24565, Accuracy:  95.3%
Epoch:     0, Process:   263/468, Loss: 0.29271, Accuracy:  91.4%
Epoch:     0, Process:   264/468, Loss: 0.37047, Accuracy:  86.7%
Epoch:     0, Process:   265/468, Loss: 0.41658, Accuracy:  89.1%
Epoch:     0, Process:   266/468, Loss: 0.43992, Accuracy:  89.8%
Epoch:     0, Process:   267/468, Loss: 0.25746, Accuracy:  93.8%
Epoch:    

Epoch:     0, Process:   386/468, Loss: 0.21357, Accuracy:  93.0%
Epoch:     0, Process:   387/468, Loss: 0.25574, Accuracy:  93.8%
Epoch:     0, Process:   388/468, Loss: 0.19779, Accuracy:  93.0%
Epoch:     0, Process:   389/468, Loss: 0.23027, Accuracy:  94.5%
Epoch:     0, Process:   390/468, Loss: 0.30081, Accuracy:  93.8%
Epoch:     0, Process:   391/468, Loss: 0.31452, Accuracy:  92.2%
Epoch:     0, Process:   392/468, Loss: 0.38159, Accuracy:  89.1%
Epoch:     0, Process:   393/468, Loss: 0.30094, Accuracy:  89.8%
Epoch:     0, Process:   394/468, Loss: 0.22085, Accuracy:  93.8%
Epoch:     0, Process:   395/468, Loss: 0.29450, Accuracy:  92.2%
Epoch:     0, Process:   396/468, Loss: 0.26503, Accuracy:  91.4%
Epoch:     0, Process:   397/468, Loss: 0.25699, Accuracy:  93.0%
Epoch:     0, Process:   398/468, Loss: 0.25778, Accuracy:  93.0%
Epoch:     0, Process:   399/468, Loss: 0.21189, Accuracy:  93.8%
Epoch:     0, Process:   400/468, Loss: 0.16426, Accuracy:  94.5%
Epoch:    

# Model 성능 테스트

In [7]:
for images, labels in test_dataset:
    accuracy = accuracy_fn(model, images, labels)
    print('Accuracy: {:5.1f}%'.format(accuracy))

Accuracy:  92.5%
